In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn
import os

import tensorflow as tf
import keras as keras
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense

from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from numpy import reshape




In [2]:
# Ładowanie daych uczących
dataset = pd.read_csv("TrainData_x.csv", sep=';')
dataset

,Fluid_level_tank102,Pressure_tank103,Pump_speed,Flowmeter,tank102_set,Pressure_set,Flow_set,Fluid_lvl_tank101_IO_link,Fluid_lvl_tank102_IO_link,Fluid_lvl_MAX_tank101,...,Work,State,Liquid_growth_B102,Pipe_clogging_factor,Flow_diff,Pressure_diff,Pressure_flow_ratio,Pressure_pumpspeed_ratio,Tanks_level_diff,Fluid_level_dependancy
0,79.25,154.83,71.46,0.00,0,150,0.0,105,80,0,...,1,2,0.00,0.0,0.00,-4.83,0.00,11064.80,25.75,0.0
1,79.09,155.59,71.08,0.00,0,150,0.0,105,80,0,...,1,2,0.00,0.0,0.00,-5.59,0.00,11059.69,25.91,0.0
2,79.25,149.47,70.71,0.00,0,150,0.0,105,80,0,...,1,2,0.00,0.0,0.00,0.53,0.00,10569.08,25.75,0.0
3,79.25,147.43,70.40,0.00,0,150,0.0,105,80,0,...,1,2,0.00,0.0,0.00,2.57,0.00,10378.78,25.75,0.0
4,79.09,150.24,70.36,0.00,0,150,0.0,105,80,0,...,1,2,0.00,0.0,0.00,-0.24,0.00,10571.38,25.91,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24661,81.61,6.89,0.00,-0.36,0,0,0.0,100,85,0,...,1,9,-0.18,0.0,0.36,-6.89,-2.48,0.00,18.39,0.0
24662,80.94,7.14,0.00,-0.34,0,0,0.0,100,85,0,...,1,9,-0.17,0.0,0.34,-7.14,-2.43,0.00,19.06,0.0
24663,80.77,7.14,0.00,-0.35,0,0,0.0,105,85,0,...,1,9,-0.18,0.0,0.35,-7.14,-2.50,0.00,24.23,0.0
24664,80.60,7.14,0.00,-0.35,0,0,0.0,105,85,0,...,1,9,-0.18,0.0,0.35,-7.14,-2.50,0.00,24.40,0.0


In [3]:
# Ładownaie danych z informacją o klasach
label = pd.read_csv("TrainData_y.csv",sep=';')
label

,State_0,Failure_F1,Failure_F2,Failure_F3,Failure_F4
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
24661,1,0,0,0,0
24662,1,0,0,0,0
24663,1,0,0,0,0
24664,1,0,0,0,0


In [4]:
# Ładowanie danych walidacyjnych
testdataset = pd.read_csv("ValidateData_x.csv",sep=';')
pd.set_option('display.max_columns', 32)
testdataset

,Fluid_level_tank102,Pressure_tank103,Pump_speed,Flowmeter,tank102_set,Pressure_set,Flow_set,Fluid_lvl_tank101_IO_link,Fluid_lvl_tank102_IO_link,Fluid_lvl_MAX_tank101,Fluid_lvl_MIN_tank101,V101,V102,V103,V104,V106,V107,V108,V109,V112,cycle_counter,EStop,Work,State,Liquid_growth_B102,Pipe_clogging_factor,Flow_diff,Pressure_diff,Pressure_flow_ratio,Pressure_pumpspeed_ratio,Tanks_level_diff,Fluid_level_dependancy
0,79.42,197.94,77.61,0.00,0,150,0.0,105,80,0,0,0,0,1,0,0,0,1,0,0,28,0,1,2,0.00,0.0,0.00,-47.94,0.00,15361.37,25.58,0.0
1,79.42,174.21,73.57,0.00,0,150,0.0,105,80,0,0,0,0,1,0,0,0,1,0,0,28,0,1,2,0.00,0.0,0.00,-24.21,0.00,12816.25,25.58,0.0
2,79.42,159.93,71.73,0.00,0,150,0.0,105,80,0,0,0,0,1,0,0,0,1,0,0,28,0,1,2,0.00,0.0,0.00,-9.93,0.00,11471.88,25.58,0.0
3,79.42,154.83,71.00,0.00,0,150,0.0,105,80,0,0,0,0,1,0,0,0,1,0,0,28,0,1,2,0.00,0.0,0.00,-4.83,0.00,10992.74,25.58,0.0
4,79.42,156.36,70.28,0.00,0,150,0.0,105,80,0,0,0,0,1,0,0,0,1,0,0,28,0,1,2,0.00,0.0,0.00,-6.36,0.00,10988.37,25.58,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,81.61,7.14,0.00,-0.35,0,0,0.0,100,85,0,0,1,1,0,0,0,1,0,0,1,8,0,1,9,-0.18,0.0,0.35,-7.14,-2.50,0.00,18.39,0.0
3604,81.27,7.14,0.00,-0.34,0,0,0.0,105,85,0,0,1,1,0,0,0,1,0,0,1,8,0,1,9,-0.17,0.0,0.34,-7.14,-2.43,0.00,23.73,0.0
3605,80.94,6.89,0.00,-0.35,0,0,0.0,105,85,0,0,1,1,0,0,0,1,0,0,1,8,0,1,9,-0.18,0.0,0.35,-6.89,-2.41,0.00,24.06,0.0
3606,80.60,7.14,0.00,-0.35,0,0,0.0,105,85,0,0,1,1,0,0,0,1,0,0,1,8,0,1,9,-0.18,0.0,0.35,-7.14,-2.50,0.00,24.40,0.0


In [5]:
# Ładowanie danych z informacją o klasie danych walidacyjnych
testlabel = pd.read_csv("ValidateData_y.csv",sep=';')
testlabel

,State_0,Failure_F1,Failure_F2,Failure_F3,Failure_F4
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
3603,1,0,0,0,0
3604,1,0,0,0,0
3605,1,0,0,0,0
3606,1,0,0,0,0


In [6]:
# Przypisywanie danych
x_train = dataset
y_train = label
x_test = testdataset
y_test = testlabel

In [7]:
# Skalowanie dancyh
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)


In [8]:
# Sprawdzenie kształtu danych
print(x_train.shape)

(24666, 32)


In [9]:
# Inicjacja modelu
model = Sequential()

In [10]:
# Dodanie warstwy wejsciowej
model.add(Dense(32,input_shape = [32,], activation='sigmoid'))

In [11]:
# Dodanie warstwy ukrytej
model.add(Dense(16, activation='sigmoid'))

In [12]:
# Dodanie warstwy wyjściowej
model.add(Dense(5,activation='softmax'))

In [13]:
# Kompilacja modelu
model.compile(
optimizer= keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07), 
loss= keras.losses.CategoricalCrossentropy(
    from_logits=False,
    #label_smoothing=0.0,
    #axis=-1,
),
metrics= ['accuracy'])

In [14]:
# Informacje o modelu
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 5)                 85        
                                                                 
Total params: 1,669
Trainable params: 1,669
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Uczenie modelu
history =model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=32)

Epoch 1/32
771/771 [==============================] - 2s 2ms/step - loss: 0.6128 - accuracy: 0.8075 - val_loss: 0.3188 - val_accuracy: 0.9202
Epoch 2/32
771/771 [==============================] - 1s 1ms/step - loss: 0.2624 - accuracy: 0.9230 - val_loss: 0.2133 - val_accuracy: 0.9271
Epoch 3/32
771/771 [==============================] - 1s 1ms/step - loss: 0.2030 - accuracy: 0.9240 - val_loss: 0.1658 - val_accuracy: 0.9324
Epoch 4/32
771/771 [==============================] - 1s 1ms/step - loss: 0.1685 - accuracy: 0.9297 - val_loss: 0.1328 - val_accuracy: 0.9482
Epoch 5/32
771/771 [==============================] - 1s 1ms/step - loss: 0.1466 - accuracy: 0.9415 - val_loss: 0.1117 - val_accuracy: 0.9460
Epoch 6/32
771/771 [==============================] - 1s 1ms/step - loss: 0.1329 - accuracy: 0.9455 - val_loss: 0.1004 - val_accuracy: 0.9526
Epoch 7/32
771/771 [==============================] - 1s 1ms/step - loss: 0.1235 - accuracy: 0.9475 - val_loss: 0.0861 - val_accuracy: 0.9678
Epoch 

In [16]:
# Ostateczny winik modelu z wartości walidacyjnych
model.evaluate(x_test, y_test, verbose=2)


113/113 - 0s - loss: 0.0543 - accuracy: 0.9787 - 145ms/epoch - 1ms/step


[0.05433829873800278, 0.9786585569381714]

In [17]:
# Tabela przedstawiająca zmianę 'loss' i 'accuracy' w epokach
metrics = pd.DataFrame(history.history)
metrics

,loss,accuracy,val_loss,val_accuracy
0,0.612761,0.807549,0.318750,0.920177
1,0.262414,0.922971,0.213262,0.927106
2,0.202991,0.923984,0.165801,0.932373
3,0.168465,0.929660,0.132757,0.948171
4,0.146583,0.941458,0.111733,0.945953
5,0.132923,0.945512,0.100441,0.952605
6,0.123541,0.947539,0.086058,0.967849
7,0.116096,0.951674,0.077936,0.972007
8,0.109783,0.953620,0.078412,0.968404
9,0.105002,0.956134,0.071777,0.975055


In [18]:
print('Wykres danych z procesu uczenia modelu')
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=metrics['loss'],line=dict(color='blue'), name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['val_loss'],line=dict(color='yellow'), name='val_loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['accuracy'],line=dict(color='blue'), name='accuracy'), row=2, col=1)
fig.add_trace(go.Scatter(y=metrics['val_accuracy'],line=dict(color='yellow'), name='val_accuracy'), row=2, col=1)
fig.update_layout(width=1200)

Wykres danych z procesu uczenia modelu


In [19]:
# Wyswietlenie określonych wartosci przez model
x_pred=model.predict(x_test)
print(y_test)
print(x_pred)

113/113 [==============================] - 0s 1ms/step
      State_0  Failure_F1  Failure_F2  Failure_F3  Failure_F4
0           1           0           0           0           0
1           1           0           0           0           0
2           1           0           0           0           0
3           1           0           0           0           0
4           1           0           0           0           0
...       ...         ...         ...         ...         ...
3603        1           0           0           0           0
3604        1           0           0           0           0
3605        1           0           0           0           0
3606        1           0           0           0           0
3607        1           0           0           0           0

[3608 rows x 5 columns]
[[9.99778569e-01 4.51004496e-08 8.55319740e-05 1.14260525e-04
  2.16105127e-05]
 [9.99761045e-01 4.79683386e-08 7.90048289e-05 1.37649346e-04
  2.22911804e-05]
 [9.99735534e-01

In [20]:
# Wyciągniecie klasy z modelu
y_pred=np.argmax(x_pred,axis=1)
y_pred


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
# Zestawienie w tabeli danej klasy oraz klasy przewidzianej przez model
pred= pd.concat([pd.DataFrame(y_test, columns=['State_0','Failure_F1','Failure_F2','Failure_F3','Failure_F4']), pd.DataFrame(y_pred, columns=['y_pred'])], axis=1)
pred.head(10)

,State_0,Failure_F1,Failure_F2,Failure_F3,Failure_F4,y_pred
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
5,1,0,0,0,0,0
6,1,0,0,0,0,0
7,1,0,0,0,0,0
8,1,0,0,0,0,0
9,1,0,0,0,0,0


In [22]:
# Konwersja danych w celu stworzenia macierzy pomyłek
yt_test = y_test.to_numpy()

In [23]:
# Wyciągnięcie danych
rounded_test=np.argmax(yt_test, axis=1)

In [24]:
# Stworzenie macierzy pomyłek
cn = confusion_matrix(rounded_test,y_pred)
print(cn)
print('dokladnosc calkowita:')
accuracy_score(rounded_test,y_pred)

[[2561    0   16    2    0]
 [   0  380    0    0    0]
 [  56    0  408    0    0]
 [   2    0    0  128    0]
 [   1    0    0    0   54]]
dokladnosc calkowita:


0.9786585365853658

In [25]:
# Obliczenie dokładnosci danych klas
cn0=cn[0,0] + cn[0,1]+ cn[0,2]+ cn[0,3]+ cn[0,4]
cn00 = cn[0,0]/cn0
cn1 = cn[1,0] + cn[1,1]+ cn[1,2]+ cn[1,3]+ cn[1,4]
cn11 = cn[1,1]/cn1
cn2 = cn[2,0] + cn[2,1]+ cn[2,2]+ cn[2,3]+ cn[2,4]
cn22 = cn[2,2]/cn2
cn3 = cn[3,0] + cn[3,1]+ cn[3,2]+ cn[3,3]+ cn[3,4]
cn33 = cn[3,3]/cn3
cn4 = cn[4,0] + cn[4,1]+ cn[4,2]+ cn[4,3]+ cn[4,4]
cn44 = cn[4,4]/cn4

In [26]:
print('Dokladnosc klasy 0, Brak uszkodzeń')
print(cn00)
print('Dokladnosc klasy 1, Korzuch')
print(cn11)
print('Dokladnosc klasy 2, Wyciek')
print(cn22)
print('Dokladnosc klasy 3, Cyber atak')
print(cn33)
print('Dokladnosc klasy 4, Przytkanie rur')
print(cn44)

Dokladnosc klasy 0, Brak uszkodzeń
0.9930205506010081
Dokladnosc klasy 1, Korzuch
1.0
Dokladnosc klasy 2, Wyciek
0.8793103448275862
Dokladnosc klasy 3, Cyber atak
0.9846153846153847
Dokladnosc klasy 4, Przytkanie rur
0.9818181818181818


In [27]:
# Zapisanie modelu
model.save('FeedForward.h5')

In [28]:
#y_pred
#pd.DataFrame(y_pred).to_csv('Predykcja_FEEDForward', sep=';', header=0,index=0)